In [1]:
# Name of your lakehouse
lakehouse_name = "lakehouse01"

# Create ProductAudit_Source with sample data
spark.sql(f"""
CREATE OR REPLACE TABLE {lakehouse_name}.ProductAudit_Source AS
SELECT 501 AS ProductID, 'Reviewed' AS ReviewFlag, DATE('2025-08-01') AS LastReviewed
UNION ALL
SELECT 502, 'Reviewed', DATE('2025-08-02')
""")

# Create ProductAudit_Target with existing data
spark.sql(f"""
CREATE OR REPLACE TABLE {lakehouse_name}.ProductAudit_Target AS
SELECT 501 AS ProductID, 'Reviewed' AS ReviewFlag, DATE('2025-07-25') AS LastReviewed
UNION ALL
SELECT 503, 'Reviewed', DATE('2025-07-15')
UNION ALL
SELECT 504, 'Reviewed', DATE('2025-07-30')
""")



StatementMeta(, de5fdff8-9a71-402f-8537-6f8f906fdd1d, 3, Finished, Available, Finished)

DataFrame[]

In [2]:
from delta.tables import DeltaTable

targetDelta = DeltaTable.forName(spark, "lakehouse01.ProductAudit_Target")
sourceDF = spark.read.table("lakehouse01.ProductAudit_Source")

(
    targetDelta.alias("targetDF")
    .merge(
        sourceDF.alias("sourceDF"),
        "sourceDF.ProductID = targetDF.ProductID"
    )

    .whenMatchedUpdate(
        set = {"LastReviewed": "sourceDF.LastReviewed"}
    )

    .whenNotMatchedBySourceUpdate(
        condition = "targetDF.LastReviewed < (current_date() - INTERVAL '10' DAY)",
        set = {"ReviewFlag": "'Expired'"}
    )

    .execute()
)

StatementMeta(, 5c1f3ebe-5a43-4b08-9869-3c2afb10b017, 4, Finished, Available, Finished)